# Ranking e Modelo Vetorial

<h3>Autora: Dandara Sousa </h3>

Essa é a resolução da segunda parte do Lab 01 da disciplina *Recuperação da Informação e Busca na Web*. 
A atividade consiste em reconstruir o índice considerando o novo dataset, refinar o índice invertido de forma a incluir o IDF (inverse document frequency) de cada termo do dicionário e o TF (term frequency) de cada termo em cada documento da lista de postings respectiva, instanciar o modelo vetorial com representação binária, TF, TF-IDF e BM25 e, por último, excutar os algoritmos separadamente em consultas específicas (veja abaixo) e retorne os top-5 documentos mais similares à cada consulta.

As consultas específicas são:
* segundo turno;
* lava jato;
* projeto de lei;
* compra de voto.
* ministério público.

A busca pelos termos deve ser conjuntiva (utilizando o AND).

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
from collections import defaultdict
import itertools
import operator

    Em primeiro momento vamos conhecer os dados que consiste em um .csv de notícias do Estadão. Temos no arquivo acesso ao timestamp, título da notícia, subtítulo da notícias, o conteúdo da notícia, a url e, por último, o id.


In [2]:
df = pd.read_csv("estadao_noticias_eleicao.csv", encoding='utf-8')
gabarito = pd.read_csv("gabarito.csv", encoding='utf-8')
df = df.replace(np.nan, '', regex = True)
df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


Para que as buscas sejam efetuadas utilizando o algoritmo de índices invertidos é necessário que tokens sejam gerados. Assim haverá uma ligação sobre qual token (no nosso caso, as palavras das notícias) aparece em qual notícia através do id desta notícia.

In [3]:
def term_freq(term, doc):
    #freq = 0
    #for word in doc:
     #   if(term == word):
      #      freq += 1
    #return freq   
    return len(list(filter((lambda x: x == term), doc)))

In [6]:
def gera_tokens(df):
    indexes = {}
    for index,row in df.iterrows():
        title_tokens = (word.lower() for word in (nl.word_tokenize(row['titulo'])))
        subtitle_tokens = (word.lower() for word in (nl.word_tokenize(row['subTitulo'])))
        notice_tokens = (word.lower() for word in (nl.word_tokenize(row['conteudo'])))
        
        tokens = list(title_tokens) +  list(subtitle_tokens) + list(notice_tokens)
        
        for token in tokens:
            if (token in indexes):
                indexes[token][token][row['idNoticia']] = term_freq(token,tokens)
            else:
                indexes[token] = {token: {row['idNoticia'] : term_freq(token,tokens)}}
            
    return indexes

In [7]:
i_indexes = gera_tokens(df)

<h4>Representação binária</h4>

Na representação binária a ideia é simples. O termo tem um score em que é somado 1 para cada vez que ele aparecer em um documento.

In [8]:
def binary_rep(query):
    bin_dict = {}
    for term in query:
        for doc_id in i_indexes[term][term].keys():
            if doc_id in bin_dict:
                bin_dict[doc_id] += 1
            else:
                bin_dict[doc_id] = 1
    return bin_dict

<h4>Frequência de termo</h4>

Agora, visualizaremos a frequência de cada termo dada uma pesquisa. A ideia é pesar mais a frequência que o termo aparece no documento e assim conseguir resolver problemas de empates que são comuns com a representação binária.


In [9]:
def term_freq(query):
    termf_dict = {}
    for term in query:
        for doc_id in i_indexes[term][term].keys():
            if (doc_id in termf_dict):
                termf_dict[doc_id] += i_indexes[term][term][doc_id]
            else:
                termf_dict[doc_id] = i_indexes[term][term][doc_id]
    return termf_dict

<h4> Frequência de termo - Frequência de documento invertida</h4>

O IDF é calculado com base na relação entre o número de documentos em que o termo aparece e o número total de documentos. 
Isso diminui a eficiência de casos onde um termo é usado diversas vezes com o único propósito de subir no ranking.

In [10]:
def term_freq_idf(query):
    m = df.size 
    term_freq_idf_dict = {}
    for term in query:
        for doc_id in i_indexes[term][term].keys():
            l = len(i_indexes[term][term])
            if (doc_id in  term_freq_idf_dict):               
                 term_freq_idf_dict[doc_id] += np.log((m+1)/l)
            else:
                term_freq_idf_dict[doc_id] = np.log((m+1)/l)
    
    termf_dict = term_freq(query)
    
    for doc_id in termf_dict.keys():
        term_freq_idf_dict[doc_id] *= termf_dict[doc_id]
    
    return term_freq_idf_dict

<h4>BM25</h4>

Esse é o algoritmo que melhor lida com repetições abusivas de um termo. Ele considera previamente a quantidade de documentos onde o termo aparece, a frequência com que ele aparece e a quantidade de documentos no total.

In [11]:
def bm25(query):
    m = df.size 
    bm25_dict = {}
    for term in query:
        for doc_id in i_indexes[term][term].keys():
            l = len(i_indexes[term][term])
            c = i_indexes[term][term][doc_id]
            if (doc_id in bm25_dict):               
                bm25_dict[doc_id] += (((l+1)*c)/(c+l))*np.log((m+1)/l)
            else:
                bm25_dict[doc_id] = (((l+1)*c)/(c+l))*np.log((m+1)/l)
    return bm25_dict

<h4>Busca</h4>

Agora, definindo a busca geral utilizando os algoritmos:

In [12]:
def search(query, algorithm):
    query = query.split()
    scores = {}
    if (algorithm == "binario"):
        scores = binary_rep(query)
    elif (algorithm == "tf"):
        scores = term_freq(query)
    elif (algorithm == "tfidf"):
        scores = term_freq_idf(query)
    elif (algorithm == "bm25"):
        scores = bm25(query)
    else:
        raise Exception("Algoritmo fora do escopo")
    
    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1), reverse = True)
    
    ids = []
    for i in range(5):
        ids.append(sorted_scores[i][0])

    return ids
    

<h4>Definindo a medição de acertos</h4>
Com base nesse [repositório](https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py).

In [13]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if (len(predicted)>k):
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if (p in actual and p not in predicted[:i]):
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

<h4>Consultas</h4>

Agora, vamos realizar consultas para os seguintes termos:
* segundo turno;
* lava jato;
* projeto de lei;
* compra de voto.
* ministério público.

In [21]:
queries = ["segundo turno", "lava jato", "projeto de lei",
           "compra de voto", "ministério público"]

result = pd.DataFrame(columns = ["consulta", "binario", "tf", "tfidf", "bm25"])

index = 0
for q in queries:
    result.loc[index] = [q, 
                          str(search(q, "binario")),
                          str(search(q, "tf")),
                          str(search(q, "tfidf")),
                          str(search(q, "bm25"))]
    index = index + 1

Antes de comparar as buscas precisamos converter para inteiro todos os elementos da lista.

In [26]:
import re
def converte_lista_int(lista):
    n_lista = []
    
    for query_result in lista:        
        as_str = re.sub('[,\[\]]', '', query_result)
        as_list = as_str.split(" ")
        list_of_int = list(map(int, as_list))
        n_lista.append(list_of_int)
    
    return n_lista

In [32]:
gab_bin = converte_lista_int(gabarito.busca_binaria)
real_bin = converte_lista_int(result.binario)
gab_tf = converte_lista_int(gabarito.tf)
real_tf = converte_lista_int(result.tf)
gab_tfidf = converte_lista_int(gabarito.tfidf)
real_tfidf = converte_lista_int(result.tfidf)
gab_bm25 = converte_lista_int(gabarito.bm25)
real_bm25 = converte_lista_int(result.bm25)
real_google = converte_lista_int(gabarito.google)


print("Para os algoritmos feitos nesse lab")
print("Modelo binário: ", mapk(gab_bin, real_bin, k = 5))
print("Modelo TF: ", mapk(gab_tf, real_tf, k = 5))
print("Modelo TF-IDF: ", mapk(gab_tfidf, real_tfidf, k = 5))
print("Modelo BM25: ", mapk(gab_bm25, real_bm25, k = 5))
print("Para algoritmos em consultas Google")
print("Modelo binário: ", mapk(real_google, real_bin, k = 5))
print("Modelo TF: ", mapk(real_google, real_tf, k = 5))
print("Modelo TF-IDF: ", mapk(real_google, real_tfidf, k = 5))
print("Modelo BM25: ", mapk(real_google, real_bm25, k = 5))

Para os algoritmos feitos nesse lab
Modelo binário:  0.24
Modelo TF:  0.71
Modelo TF-IDF:  0.6046666666666667
Modelo BM25:  0.5519999999999999
Para algoritmos em consultas Google
Modelo binário:  0.04
Modelo TF:  0.048
Modelo TF-IDF:  0.048
Modelo BM25:  0.048


Com isso, nosso melhor modelo é o TF com a maior taxa de precisão de acertos. Em seguida temos o TF-IDF, o BM25 e por último com uma grande diferença de precisão o binário. Podemos ver que em consulas Google todos se saem iguais exceto o modelo binário. 
De forma geral o modelo binário é o mais simples e é de se esperar que tenha menos acertos que os outros. 